# Project 5

Chen Liang

Project 5 is an improvement based on project 4. In this project, the structure of the model is changed to handle temporal information more effectively. The way how input data is preprocess is also changed in correspondance to the model structure.

In [1]:
from os import listdir
from os.path import isfile, join
import cv2
import numpy as np
import random
from sklearn.utils import shuffle
from joblib import Parallel, delayed
from video_to_numpy import video_to_numpy
import numpy as np
import matplotlib.pyplot as plt
import scipy
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from scipy import stats
import sklearn.metrics as metrices
import seaborn as sns
from tensorflow.keras.utils import to_categorical

## Data Preprocessing

Different from project 4 where several adjacent images are combined to a single image to capture changes, this project images are just frames of videos, and they are then combined into small blocks to be used by time distributed CNN layers.

### Generate Image Arrays

In [2]:
fpath='hmdb/'
punch_fnames=[join(fpath+'punch/', f) for f in listdir(fpath+'punch/') if isfile(join(fpath+'punch/', f))]
#notpunch_fnames=[join(fpath+'notpunch/', f) for f in listdir(fpath+'notpunch/') if isfile(join(fpath+'notpunch/', f))]
notpunch_fnames=[join(fpath+'mess/', f) for f in listdir(fpath+'mess/') if isfile(join(fpath+'mess/', f))]

In [106]:
num_processes = 12
_=Parallel(n_jobs=num_processes)(delayed(video_to_numpy)\
                               (str(video_id),'hframes/punch/',temporal_dimension=15) for video_id in punch_fnames)

In [107]:
num_processes = 12
_=Parallel(n_jobs=num_processes)(delayed(video_to_numpy)\
                               (str(video_id),'hframes/notpunch/',temporal_dimension=15)\
                                 for video_id in notpunch_fnames)

### Organize Images

In [3]:
frame_path='hframes/'
punch_imgs=[join(frame_path+'punch/', f) for f in listdir(frame_path+'punch/') if isfile(join(frame_path+'punch/', f))]
notpunch_imgs=[join(frame_path+'notpunch/', f) for f in listdir(frame_path+'notpunch/') if isfile(join(frame_path+'notpunch/', f))]

In [4]:
print('punch imgs: ',len(punch_imgs),'. not_punch imgs: ',len(notpunch_imgs))

punch imgs:  120 . not_punch imgs:  100


In [5]:
Xt=None
ypc=0
ync=0
for i in punch_imgs[:30]:
    Xtmp=np.load(i)
    if Xt is None:
        Xt=Xtmp
    else:
        Xt=np.concatenate([Xt,Xtmp])
    ypc+=Xtmp.shape[0]
for i in notpunch_imgs[:30]:
    Xtmp=np.load(i)
    if Xt is None:
        Xt=Xtmp
    else:
        Xt=np.concatenate([Xt,Xtmp])
    ync+=Xtmp.shape[0]

In [6]:
yt=np.array([1]*ypc+[0]*ync)

In [7]:
Xt.shape

(6053, 15, 224, 224, 3)

In [8]:
yt.shape

(6053,)

In [9]:
Xtv=None
ypcv=0
yncv=0
for i in punch_imgs[40:45]:
    Xtmp=np.load(i)
    if Xtv is None:
        Xtv=Xtmp
    else:
        Xtv=np.concatenate([Xtv,Xtmp])
    ypcv+=Xtmp.shape[0]
for i in notpunch_imgs[40:45]:
    Xtmp=np.load(i)
    if Xtv is None:
        Xtv=Xtmp
    else:
        Xtv=np.concatenate([Xtv,Xtmp])
    yncv+=Xtmp.shape[0]

In [10]:
ytv=np.array([1]*ypcv+[0]*yncv)

In [11]:
Xtv.shape

(422, 15, 224, 224, 3)

In [12]:
ytv.shape

(422,)

In [13]:
Xt, yt= shuffle(Xt, yt)
Xtv, ytv= shuffle(Xtv, ytv)

In [21]:
cv2.imshow('qwe',Xt[3000,0,:,:,:])
cv2.waitKey(0)
cv2.destroyAllWindows()

In [24]:
ytv

array([0, 0, 0, ..., 0, 1, 0])

In [25]:
ytb = to_categorical(yt)
ytvb = to_categorical(ytv)

In [26]:
ytvb

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [1., 0.],
       [0., 1.],
       [1., 0.]], dtype=float32)

## Model Training

In [16]:
input_shape=(15, 224, 224, 3)

In [65]:
  def create_model(input_shape):
        def add_default_block(model, kernel_filters, init, reg_lambda):

            # conv
            model.add(layers.TimeDistributed(layers.Conv2D(kernel_filters, (3, 3), padding='same',
                                             kernel_initializer=init, kernel_regularizer=tf.keras.regularizers.l2(l=reg_lambda))))
            model.add(layers.TimeDistributed(layers.BatchNormalization()))
            model.add(layers.TimeDistributed(layers.Activation('relu')))
            # conv
            model.add(layers.TimeDistributed(layers.Conv2D(kernel_filters, (3, 3), padding='same',
                                             kernel_initializer=init, kernel_regularizer=tf.keras.regularizers.l2(l=reg_lambda))))
            model.add(layers.TimeDistributed(layers.BatchNormalization()))
            model.add(layers.TimeDistributed(layers.Activation('relu')))
            # max pool
            model.add(layers.TimeDistributed(layers.MaxPooling2D((2, 2), strides=(2, 2))))

            return model

        initialiser = 'glorot_uniform'
        reg_lambda  = 0.001

        model = models.Sequential()

        # first (non-default) block
        model.add(layers.TimeDistributed(layers.Conv2D(32, (7, 7), strides=(2, 2), padding='same',
                                         kernel_initializer=initialiser, kernel_regularizer=tf.keras.regularizers.l2(l=reg_lambda)),
                                  input_shape=input_shape))
        model.add(layers.TimeDistributed(layers.BatchNormalization()))
        model.add(layers.TimeDistributed(layers.Activation('relu')))
        model.add(layers.TimeDistributed(layers.Conv2D(32, (3,3), kernel_initializer=initialiser, kernel_regularizer=tf.keras.regularizers.l2(l=reg_lambda))))
        model.add(layers.TimeDistributed(layers.BatchNormalization()))
        model.add(layers.TimeDistributed(layers.Activation('relu')))
        model.add(layers.TimeDistributed(layers.MaxPooling2D((2, 2), strides=(2, 2))))

        # 2nd-5th (default) blocks
        model = add_default_block(model, 64,  init=initialiser, reg_lambda=reg_lambda)
        model = add_default_block(model, 128, init=initialiser, reg_lambda=reg_lambda)
        model = add_default_block(model, 256, init=initialiser, reg_lambda=reg_lambda)
        model = add_default_block(model, 512, init=initialiser, reg_lambda=reg_lambda)

        # LSTM output head
        model.add(layers.TimeDistributed(layers.Flatten()))
        model.add(layers.LSTM(256, return_sequences=False, dropout=0.5))
        model.add(layers.Dense(2, activation='softmax'))

        return model

In [17]:
def create_model():
    model1=models.Sequential()
    model1.add(layers.TimeDistributed(layers.Conv2D(64,kernel_size=(3,3),strides=(2,2),activation='relu'),input_shape=input_shape))
    model1.add(layers.TimeDistributed(layers.MaxPooling2D((2, 2))))
    model1.add(layers.TimeDistributed(layers.Conv2D(128, (3, 3), activation='relu')))
    model1.add(layers.TimeDistributed(layers.MaxPooling2D((2, 2))))
    model1.add(layers.TimeDistributed(layers.Conv2D(128, (3, 3), activation='relu')))
    model1.add(layers.TimeDistributed(layers.MaxPooling2D((2, 2))))
    model1.add(layers.TimeDistributed(layers.Conv2D(128, (3, 3), activation='relu')))
    model1.add(layers.TimeDistributed(layers.MaxPooling2D((2, 2))))
    model1.add(layers.TimeDistributed(layers.Conv2D(128, (3, 3), activation='relu')))
    model1.add(layers.TimeDistributed(layers.MaxPooling2D((2, 2))))
    model1.add(layers.TimeDistributed(layers.Dropout(0.25)))
    model1.add(layers.TimeDistributed(layers.Flatten()))
    model1.add(layers.TimeDistributed(layers.Dense(128, activation='relu')))
    model1.add(layers.TimeDistributed(layers.Dense(2, activation='softmax')))
    model1.add(layers.LSTM(5))
    #model1.add(layers.Activation('softmax'))
    #model1.add(layers.Dense(64, activation='relu'))
    model1.add(layers.Dense(2, activation='softmax'))
    #model1.add(layers.Dense(1, activation='softmax'))
    return model1

In [75]:
  def create_model(input_shape):
        def add_default_block(model, kernel_filters, init, reg_lambda):

            # conv
            model.add(layers.TimeDistributed(layers.Conv2D(kernel_filters, (3, 3), padding='same',activation='relu',
                                             kernel_initializer=init, kernel_regularizer=tf.keras.regularizers.l2(l=reg_lambda))))
            model.add(layers.TimeDistributed(layers.BatchNormalization()))
            # conv
            model.add(layers.TimeDistributed(layers.Conv2D(kernel_filters, (3, 3), padding='same',activation='relu',
                                             kernel_initializer=init, kernel_regularizer=tf.keras.regularizers.l2(l=reg_lambda))))
            model.add(layers.TimeDistributed(layers.BatchNormalization()))
            # max pool
            model.add(layers.TimeDistributed(layers.MaxPooling2D((2, 2), strides=(2, 2))))

            return model

        initialiser = 'glorot_uniform'
        reg_lambda  = 0.001

        model = models.Sequential()

        # first (non-default) block
        model.add(layers.TimeDistributed(layers.Conv2D(32, (7, 7), strides=(2, 2), padding='same',activation='relu',
                                         kernel_initializer=initialiser, kernel_regularizer=tf.keras.regularizers.l2(l=reg_lambda)),
                                  input_shape=input_shape))
        model.add(layers.TimeDistributed(layers.BatchNormalization()))
        model.add(layers.TimeDistributed(layers.Conv2D(32, (3,3),activation='relu', kernel_initializer=initialiser, kernel_regularizer=tf.keras.regularizers.l2(l=reg_lambda))))
        model.add(layers.TimeDistributed(layers.BatchNormalization()))
        model.add(layers.TimeDistributed(layers.MaxPooling2D((2, 2), strides=(2, 2))))

        # 2nd-5th (default) blocks
        model = add_default_block(model, 64,  init=initialiser, reg_lambda=reg_lambda)
        model = add_default_block(model, 128, init=initialiser, reg_lambda=reg_lambda)
        #model = add_default_block(model, 256, init=initialiser, reg_lambda=reg_lambda)
        #model = add_default_block(model, 512, init=initialiser, reg_lambda=reg_lambda)

        # LSTM output head
        model.add(layers.TimeDistributed(layers.Flatten()))
        model.add(layers.LSTM(256, return_sequences=False, dropout=0.5))
        model.add(layers.Dense(2, activation='softmax'))

        return model

In [90]:
  def create_model(input_shape):
        def add_default_block(model, kernel_filters, init, reg_lambda):

            # conv
            model.add(layers.TimeDistributed(layers.Conv2D(kernel_filters, (3, 3), padding='same',activation='relu',
                                             kernel_initializer=init, kernel_regularizer=tf.keras.regularizers.l2(l=reg_lambda))))
            model.add(layers.TimeDistributed(layers.BatchNormalization()))
            # conv
            model.add(layers.TimeDistributed(layers.Conv2D(kernel_filters, (3, 3), padding='same',activation='relu',
                                             kernel_initializer=init, kernel_regularizer=tf.keras.regularizers.l2(l=reg_lambda))))
            model.add(layers.TimeDistributed(layers.BatchNormalization()))
            # max pool
            model.add(layers.TimeDistributed(layers.MaxPooling2D((2, 2), strides=(2, 2))))

            return model
        initialiser = 'glorot_uniform'
        reg_lambda  = 0.001

        model = models.Sequential()
        model.add(layers.TimeDistributed(layers.Conv2D(32, (7, 7), strides=(2, 2), padding='same',activation='relu',
                                         kernel_initializer=initialiser, kernel_regularizer=tf.keras.regularizers.l2(l=reg_lambda)),
                                  input_shape=input_shape))
        model.add(layers.TimeDistributed(layers.BatchNormalization()))
        model.add(layers.TimeDistributed(layers.Conv2D(32, (3,3),activation='relu', kernel_initializer=initialiser, kernel_regularizer=tf.keras.regularizers.l2(l=reg_lambda))))
        model.add(layers.TimeDistributed(layers.BatchNormalization()))
        model.add(layers.TimeDistributed(layers.MaxPooling2D((2, 2), strides=(2, 2))))

        model = add_default_block(model, 64,  init=initialiser, reg_lambda=reg_lambda)
        model = add_default_block(model, 128, init=initialiser, reg_lambda=reg_lambda)

        # LSTM output head
        model.add(layers.TimeDistributed(layers.Flatten()))
        model.add(layers.LSTM(256, return_sequences=False, dropout=0.5))
        model.add(layers.Dense(2, activation='softmax'))

        return model

In [18]:
reg_val  = 0.001
def create_model():
    

    model = models.Sequential()
    model.add(layers.TimeDistributed(layers.Conv2D(32, (7, 7), strides=(2, 2), padding='same',activation='relu',
                                     kernel_regularizer=tf.keras.regularizers.l2(l=reg_val)),
                              input_shape=input_shape))
    model.add(layers.TimeDistributed(layers.BatchNormalization()))
    model.add(layers.TimeDistributed(layers.Conv2D(32, (3,3),activation='relu', kernel_regularizer=tf.keras.regularizers.l2(l=reg_val))))
    model.add(layers.TimeDistributed(layers.BatchNormalization()))
    model.add(layers.TimeDistributed(layers.MaxPooling2D((2, 2), strides=(2, 2))))
    model.add(layers.TimeDistributed(layers.Conv2D(64, (3, 3), padding='same',activation='relu',
                                      kernel_regularizer=tf.keras.regularizers.l2(l=reg_val))))
    model.add(layers.TimeDistributed(layers.BatchNormalization()))
    model.add(layers.TimeDistributed(layers.Conv2D(64, (3, 3), padding='same',activation='relu',
                                      kernel_regularizer=tf.keras.regularizers.l2(l=reg_val))))
    model.add(layers.TimeDistributed(layers.BatchNormalization()))
    model.add(layers.TimeDistributed(layers.MaxPooling2D((2, 2), strides=(2, 2))))
    model.add(layers.TimeDistributed(layers.Conv2D(128, (3, 3), padding='same',activation='relu',
                                      kernel_regularizer=tf.keras.regularizers.l2(l=reg_val))))
    model.add(layers.TimeDistributed(layers.BatchNormalization()))
    model.add(layers.TimeDistributed(layers.Conv2D(128, (3, 3), padding='same',activation='relu',
                                      kernel_regularizer=tf.keras.regularizers.l2(l=reg_val))))
    model.add(layers.TimeDistributed(layers.BatchNormalization()))
    model.add(layers.TimeDistributed(layers.MaxPooling2D((2, 2), strides=(2, 2))))
    model.add(layers.TimeDistributed(layers.Flatten()))
    model.add(layers.LSTM(256, dropout=0.5))
    model.add(layers.Dense(2, activation='softmax'))

    return model

In [24]:
model=create_model()
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              #loss='hinge',
              metrics=['accuracy'])

In [25]:
history = model.fit(Xt, yt, epochs=15, batch_size=16,
                    validation_data=(Xtv, ytv))

Train on 6053 samples, validate on 422 samples
Epoch 1/15
6053/6053 [==============================] - 125s 21ms/sample - loss: 0.3754 - accuracy: 0.9803 - val_loss: 1.6606 - val_accuracy: 0.5379
Epoch 2/15
6053/6053 [==============================] - 122s 20ms/sample - loss: 0.2627 - accuracy: 0.9853 - val_loss: 2.5550 - val_accuracy: 0.5190
Epoch 3/15
6053/6053 [==============================] - 122s 20ms/sample - loss: 0.1881 - accuracy: 0.9869 - val_loss: 2.3429 - val_accuracy: 0.5758
Epoch 4/15
6053/6053 [==============================] - 122s 20ms/sample - loss: 0.1152 - accuracy: 0.9955 - val_loss: 2.0036 - val_accuracy: 0.4526
Epoch 5/15
6053/6053 [==============================] - 122s 20ms/sample - loss: 0.0795 - accuracy: 0.9945 - val_loss: 3.1078 - val_accuracy: 0.5071
Epoch 6/15
6053/6053 [==============================] - 122s 20ms/sample - loss: 0.1096 - accuracy: 0.9883 - val_loss: 3.4203 - val_accuracy: 0.3981
Epoch 7/15
6053/6053 [==============================] - 122

In [28]:
model.save('model/hmdbmodelProj5')

TypeError: unsupported callable

In [30]:
model.save_weights('./model/proj5')